In [9]:
import duckdb

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("train_with_te.parquet")

In [ ]:
df.dtypes

,0
id,uint32
Brand,object
Material,object
Size,object
comp,float32
L_comp,bool
Waterproof,bool
Style,object
Color,object
wc,float32


In [ ]:
con = duckdb.connect(database=":memory:", read_only=False)
con.register("train", df)

In [ ]:
df2 = con.execute("""
WITH
params AS (SELECT 0.03::DOUBLE AS rare_pct),              -- 4% rarity threshold
base AS (
  SELECT *
  FROM df
),
aug AS (
  SELECT
    base.*,

    -- global total
    COUNT(*) OVER () AS n_total,

    -- single-key counts
    COUNT(*) OVER (PARTITION BY brand)     AS brand_count,
    COUNT(*) OVER (PARTITION BY style)     AS style_count,
    COUNT(*) OVER (PARTITION BY material)  AS material_count,
    COUNT(*) OVER (PARTITION BY size)      AS size_count,
    COUNT(*) OVER (PARTITION BY color)     AS color_count,
    COUNT(*) OVER (PARTITION BY waterproof)AS wp_count,

    -- pair counts
    COUNT(*) OVER (PARTITION BY brand, style)         AS brand_style_count,
    COUNT(*) OVER (PARTITION BY brand, material)      AS brand_material_count,
    COUNT(*) OVER (PARTITION BY brand, size)          AS brand_size_count,
    COUNT(*) OVER (PARTITION BY brand, color)         AS brand_color_count,
    COUNT(*) OVER (PARTITION BY brand, waterproof)    AS brand_wp_count,

    COUNT(*) OVER (PARTITION BY style, material)      AS style_material_count,
    COUNT(*) OVER (PARTITION BY style, size)          AS style_size_count,
    COUNT(*) OVER (PARTITION BY style, color)         AS style_color_count,
    COUNT(*) OVER (PARTITION BY style, waterproof)    AS style_wp_count,

    COUNT(*) OVER (PARTITION BY material, size)       AS material_size_count,
    COUNT(*) OVER (PARTITION BY material, color)      AS material_color_count,
    COUNT(*) OVER (PARTITION BY material, waterproof) AS material_wp_count
  FROM base
)
SELECT
  aug.*,

  -- shares within parent groups
  CAST(brand_style_count   AS DOUBLE) / NULLIF(brand_count,  0) AS brand_style_share_within_brand,
  CAST(brand_color_count   AS DOUBLE) / NULLIF(brand_count,  0) AS brand_color_share_within_brand,
  CAST(brand_material_count AS DOUBLE) / NULLIF(brand_count, 0) AS brand_material_share_within_brand,
  CAST(brand_wp_count      AS DOUBLE) / NULLIF(brand_count,  0) AS brand_wp_share_within_brand,

  CAST(style_color_count   AS DOUBLE) / NULLIF(style_count,  0) AS style_color_share_within_style,

  -- global shares for singles
  CAST(material_count AS DOUBLE) / NULLIF(n_total, 0) AS material_global_share,
  CAST(style_count    AS DOUBLE) / NULLIF(n_total, 0) AS style_global_share,
  CAST(color_count    AS DOUBLE) / NULLIF(n_total, 0) AS color_global_share,

  -- rarity flags by percentage
  (CAST(brand_color_count    AS DOUBLE) / NULLIF(brand_count,  0)) < (SELECT rare_pct FROM params) AS brand_color_is_rare,
  (CAST(brand_style_count    AS DOUBLE) / NULLIF(brand_count,  0)) < (SELECT rare_pct FROM params) AS brand_style_is_rare,
  (CAST(brand_material_count AS DOUBLE) / NULLIF(brand_count,  0)) < (SELECT rare_pct FROM params) AS brand_material_is_rare,
  (CAST(brand_wp_count       AS DOUBLE) / NULLIF(brand_count,  0)) < (SELECT rare_pct FROM params) AS brand_wp_is_rare,

  (CAST(material_count AS DOUBLE) / NULLIF(n_total, 0)) < (SELECT rare_pct FROM params) AS material_is_rare,
  (CAST(style_count    AS DOUBLE) / NULLIF(n_total, 0)) < (SELECT rare_pct FROM params) AS style_is_rare,
  (CAST(color_count    AS DOUBLE) / NULLIF(n_total, 0)) < (SELECT rare_pct FROM params) AS color_is_rare,

  -- log1p on some high-cardinality combos
  LOG(1 + brand_color_count)  AS log1p_brand_color_count,
  LOG(1 + style_color_count)  AS log1p_style_color_count
FROM aug
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df2.dtypes

,0
id,uint32
Brand,object
Material,object
Size,object
comp,float32
...,...
material_is_rare,bool
style_is_rare,bool
color_is_rare,bool
log1p_brand_color_count,float64


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df2.drop(
    columns=['brand_style_share_within_brand',
             'brand_material_share_within_brand',
             'brand_wp_share_within_brand'],
    errors='ignore',
    inplace=True
)


In [10]:
def downcast_duckdb_table(con, table_name,
                          keep_double=("Price",),   # keep price as double
                          downcast_floats=True,
                          cast_01_to_bool=True):    # ints (0/1) -> boolean
    def q(name: str) -> str:
        return '"' + name.replace('"', '""') + '"'

    def pick_int_type(minv, maxv, unsigned=False):
        if unsigned:
            if maxv <= 255:        return "UTINYINT"
            if maxv <= 65535:      return "USMALLINT"
            if maxv <= 4294967295: return "UINTEGER"
            return "UBIGINT"
        else:
            if minv >= -128 and maxv <= 127:               return "TINYINT"
            if minv >= -32768 and maxv <= 32767:           return "SMALLINT"
            if minv >= -2147483648 and maxv <= 2147483647: return "INTEGER"
            return "BIGINT"

    type_sizes = {
        "TINYINT":1, "UTINYINT":1, "SMALLINT":2, "USMALLINT":2,
        "INTEGER":4, "UINTEGER":4, "REAL":4, "FLOAT":4,
        "BIGINT":8, "UBIGINT":8, "DOUBLE":8, "HUGEINT":16
    }

    # Version-robust DESCRIBE
    sch = con.sql(f"DESCRIBE {q(table_name)}").df()
    col_field  = "column_name" if "column_name" in sch.columns else "Column"
    type_field = "column_type" if "column_type" in sch.columns else "Type"

    print(f"🚀 Optimizing: {table_name}")
    for _, row in sch.iterrows():
        col = str(row[col_field])
        typ = str(row[type_field]).upper()

        # Skip complex/nondowncast types
        if typ.startswith(("DECIMAL", "TIMESTAMP", "DATE")):
            continue

        # 0/1 -> BOOLEAN
        if cast_01_to_bool and ("INT" in typ) and "HUGEINT" not in typ:
            mn, mx = con.sql(
                f"SELECT MIN({q(col)}), MAX({q(col)}) FROM {q(table_name)}"
            ).fetchone()
            if mn is not None and mx is not None and mn in (0, None) and mx in (0,1):
                print(f"  {col}: {typ} -> BOOLEAN")
                con.sql(f"ALTER TABLE {q(table_name)} ALTER COLUMN {q(col)} SET DATA TYPE BOOLEAN")
                continue

        # Integers → smallest fitting
        if "INT" in typ and "HUGEINT" not in typ:
            mn, mx = con.sql(
                f"SELECT MIN({q(col)}), MAX({q(col)}) FROM {q(table_name)}"
            ).fetchone()
            if mn is None or mx is None:
                continue
            target = pick_int_type(mn, mx, unsigned=(mn >= 0))
            cur_sz = type_sizes.get(typ)
            tgt_sz = type_sizes.get(target)
            if cur_sz and tgt_sz and tgt_sz < cur_sz:
                print(f"  {col}: {typ} -> {target}")
                con.sql(f"ALTER TABLE {q(table_name)} ALTER COLUMN {q(col)} SET DATA TYPE {target}")
            continue

        # DOUBLE → REAL (unless whitelisted)
        if downcast_floats and typ == "DOUBLE" and col not in keep_double:
            print(f"  {col}: DOUBLE -> REAL")
            con.sql(f"ALTER TABLE {q(table_name)} ALTER COLUMN {q(col)} SET DATA TYPE REAL")

    print("✅ Done.")

In [11]:
con.register('df2_df', df2)
con.execute("CREATE OR REPLACE TABLE df2 AS SELECT * FROM df2_df")
con.unregister('df2_df')

# Now downcast the DuckDB table
downcast_duckdb_table(
    con, 'df2',
    keep_double=("Price",),
    downcast_floats=True,
    cast_01_to_bool=True
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🚀 Optimizing: df2
  brand_price_te: DOUBLE -> REAL
  material_price_te: DOUBLE -> REAL
  size_price_te: DOUBLE -> REAL
  style_price_te: DOUBLE -> REAL
  waterproof_price_te: DOUBLE -> REAL
  brand_material_price_te: DOUBLE -> REAL
  brand_size_price_te: DOUBLE -> REAL
  material_size_price_te: DOUBLE -> REAL
  brand_price_count_te: BIGINT -> UINTEGER
  brand_material_price_count_te: BIGINT -> UINTEGER
  n_total: BIGINT -> UINTEGER
  brand_count: BIGINT -> UINTEGER
  style_count: BIGINT -> UINTEGER
  material_count: BIGINT -> UINTEGER
  size_count: BIGINT -> UINTEGER
  color_count: BIGINT -> UINTEGER
  wp_count: BIGINT -> UINTEGER
  brand_style_count: BIGINT -> UINTEGER
  brand_material_count: BIGINT -> UINTEGER
  brand_size_count: BIGINT -> UINTEGER
  brand_color_count: BIGINT -> UINTEGER
  brand_wp_count: BIGINT -> UINTEGER
  style_material_count: BIGINT -> UINTEGER
  style_size_count: BIGINT -> UINTEGER
  style_color_count: BIGINT -> UINTEGER
  style_wp_count: BIGINT -> UINTEGER
  m

In [12]:
df2.shape

(3802092, 87)

In [14]:
con.register('final', df2)

In [15]:
con.sql("""
  COPY final
  TO 'final_features_backpack.parquet'
  (FORMAT PARQUET, COMPRESSION 'ZSTD');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))